# initialization

In [1]:
%env PYSPARK_SUBMIT_ARGS=--driver-java-options -Dio.netty.tryReflectionSetAccessible=true --conf spark.executor.extraJavaOptions=-Dio.netty.tryReflectionSetAccessible=true pyspark-shell

env: PYSPARK_SUBMIT_ARGS=--driver-java-options -Dio.netty.tryReflectionSetAccessible=true --conf spark.executor.extraJavaOptions=-Dio.netty.tryReflectionSetAccessible=true pyspark-shell


In [2]:
# Local path to gluten jar.
gluten_target_jar='/opt/gluten/jars/gluten-velox-bundle-spark3.5_2.12-linux_amd64-1.5.0-SNAPSHOT.jar'

# Select workload. Can be either 'tpch' or 'tpcds'.
workload='tpch'

# Run with gluten. If False, run Spark.
run_gluten=True

# TPC tables
#tpch_tabledir='/opt/spark/database/tpch_sf10_parquet_zstd'
#tpcds_tabledir='/opt/spark/database/tpch_sf10_parquet_zstd'

tabledir = '/opt/spark/database/tpch_sf10_parquet_zstd'

# TPC queries
#tpch_query_path='/opt/spark/tpch-queries'
#tpcds_query_path='/opt/spark/tpcds-queries'

tpc_query_path = '/opt/spark/tpch-queries/'

# Parallelism
executors_per_node=2

#gluten_tpch_task_per_core=2
#gluten_tpcds_task_per_core=4
#spark_tpch_task_per_core=8
#spark_tpcds_task_per_core=8

task_per_core=2

# Offheap ratio. 0 to disable offheap for Spark.
# onheap:offheap = 1:2
#spark_offheap_ratio=2.0
# onheap:offheap = 1:7
#gluten_offheap_ratio=7.0

offheap_ratio = 7.0

# spark.io.compression.codec
spark_codec='lz4'
# spark.gluten.sql.columnar.shuffle.codec
gluten_codec='lz4'

In [3]:
%run ./native_sql_initialize.ipynb

home: /home/spark
cwd: /opt/spark/work-dir/ipython/benchmark_velox


,Environment Variable,Value
0,HOSTNAME,ccdev31.fyre.ibm.com
1,SPARK_NO_DAEMONIZE,true
2,SHARED_LIBS,/shared-libs
3,PWD,/opt/spark/work-dir
4,HOME,/home/spark
5,LANG,C.utf8
6,TERM,xterm-color
7,LESSOPEN,||/usr/bin/lesspipe.sh %s
8,SHLVL,0
9,SPARK_HOME,/opt/spark


localhost: ccdev31.fyre.ibm.com
ip: 10.51.5.211
Spark version from SPARK_HOME: 3.5.2


# Application Level Configuration

In [4]:
if run_gluten:
    sct=GlutenSparkContext(executors_per_node, task_per_core, gluten_target_jar, offheap_ratio)
    sct.conf.set('spark.gluten.sql.columnar.shuffle.codec', gluten_codec)
else:
    sct=VanillaSparkContext(executors_per_node, task_per_core, gluten_target_jar, offheap_ratio)
    sct.conf.set('spark.io.compression.codec', spark_codec)

In [5]:
if workload.lower()=="tpch":
    bm=TPCHBenchmark(sct, tabledir, 'parquet', tpc_query_path)
else:
    bm=TPCDSBenchmark(sct, tabledir, 'parquet', tpc_query_path)
bm.initialize()


            executors per node: 2
            parallelism: 32
            executor memory: 2283m
            offheap memory: 15984m
        
spark.serializer:  org.apache.spark.serializer.KryoSerializer
master:  spark://ccdev31.fyre.ibm.com:7077


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/08 23:24:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/08 23:25:01 WARN VeloxListenerApi: Memory overhead is set to 3072MiB which is smaller than the recommended size 4795MiB. This may cause OOM.
W20250408 23:25:03.618028   409 MemoryArbitrator.cpp:84] Query memory capacity[2.25GB] is set for NOOP arbitrator which has no capacity enforcement
/opt/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


appid:  app-20250408232504-0000
SparkConf:


,key,value
0,spark.eventLog.enabled,true
1,spark.app.id,app-20250408232504-0000
2,spark.app.startTime,1744154699178
3,spark.hadoop.fs.s3a.endpoint,https://s3.us-east-1.amazonaws.com
4,spark.gluten.sql.columnar.shuffle.codec,lz4
5,spark.executor.extraJavaOptions,-Djava.net.preferIPv6Addresses=false -XX:+Igno...
6,spark.cleaner.periodicGC.interval,10s
7,spark.serializer,org.apache.spark.serializer.KryoSerializer
8,spark.gluten.sql.session.timeZone.default,Etc/UTC
9,spark.driver.host,ccdev31.fyre.ibm.com


start run:  app-20250408232504-0000
http://10.51.5.211:18080/history/app-20250408232504-0000/jobs/


# Run Workload

In [ ]:
bm.test_tpc.power_run(explain=False, print_result=False, load_table=True, action=lambda df: df.collect())

Loading all tables: ['customer', 'lineitem', 'nation', 'orders', 'part', 'partsupp', 'region', 'supplier']


start query q1, application id app-20250408232504-0000
Start time : 3157642.993387116


start query q2, application id app-20250408232504-0000
Start time : 3157652.887261534


start query q3, application id app-20250408232504-0000
Start time : 3157660.859836286


start query q4, application id app-20250408232504-0000
Start time : 3157665.38484106


...

start query q14, application id app-20250408232504-0000
Start time : 3157725.289641529


start query q15, application id app-20250408232504-0000
Start time : 3157727.501044003


start query q16, application id app-20250408232504-0000
Start time : 3157731.975916747


start query q17, application id app-20250408232504-0000
Start time : 3157736.600842477


start query q18, application id app-20250408232504-0000
Start time : 3157741.868164683


In [9]:
bm.test_tpc.print_result()

{'q1': {'query_name': 'q1', 'application_id': 'app-20250408232504-0000', 'application_time_taken': 9.57947343075648, 'query_status': 'pass', 'start_time': 1744154749.8456852}, 'q2': {'query_name': 'q2', 'application_id': 'app-20250408232504-0000', 'application_time_taken': 7.913997981697321, 'query_status': 'pass', 'start_time': 1744154759.7395751}, 'q3': {'query_name': 'q3', 'application_id': 'app-20250408232504-0000', 'application_time_taken': 4.457312548067421, 'query_status': 'pass', 'start_time': 1744154767.7121496}, 'q4': {'query_name': 'q4', 'application_id': 'app-20250408232504-0000', 'application_time_taken': 4.47059272415936, 'query_status': 'pass', 'start_time': 1744154772.237155}, 'q5': {'query_name': 'q5', 'application_id': 'app-20250408232504-0000', 'application_time_taken': 9.149302499834448, 'query_status': 'pass', 'start_time': 1744154776.7635832}, 'q6': {'query_name': 'q6', 'application_id': 'app-20250408232504-0000', 'application_time_taken': 1.2650287300348282, 'que

In [10]:
bm.collect_profile()

mkdir: cannot create directory ‘/opt/spark/work-dir/profile/app-20250408232504-0000/worker1’: File exists
appid is  app-20250408232504-0000
saved to /opt/spark/work-dir/profile/


In [12]:
import gzip
with open('/opt/spark/work-dir/profile/app-20250408232504-0000/app-20250408232504-0000', 'rb') as f_in, \
    gzip.open('/opt/spark/work-dir/profile/app-20250408232504-0000/app-20250408232504-0000.gz', 'wb') as f_out:
    f_out.writelines(f_in)